In [1]:
import sys
sys.path.append('../..')
import pybeamtools.controls as pc

[INFO ] [     MainThread] [12:58:54      logging.py  23] [pybeamtools.utils.logging] Starting shared logging thread on PID 26308
[INFO ] [   log_listener] [12:58:54      logging.py  32] [pybeamtools.utils.logging] Logging listener started on PID 26308
[INFO ] [     MainThread] [12:58:54      logging.py  27] [pybeamtools.utils.logging] Logging setup finished
[INFO ] [     MainThread] [12:58:55        utils.py 148] [numexpr.utils] Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[INFO ] [     MainThread] [12:58:55        utils.py 160] [numexpr.utils] NumExpr defaulting to 8 threads.
[DEBUG] [     MainThread] [12:58:55     _backend.py  35] [caproto] Backend 'array' registered
[DEBUG] [     MainThread] [12:58:55     _backend.py  35] [caproto] Backend 'numpy' registered
[DEBUG] [     MainThread] [12:58:55     _backend.py  47] [caproto] Selecting backend: 'numpy'


In [2]:
from pybeamtools.sim.core import SimulationEngine, ChannelMap, ChannelMapper
from pybeamtools.sim.devices import RealisticMagnet, Oscillator

sim = SimulationEngine()

def read_device(device, output):
    return device.read()

def write_device(device, output, value):
    return device.write(value)

mag1 = RealisticMagnet(name='mag1', value=0, low=-20.0, high=20.0, noise=0.1, resolution=None, model='instant')
m = ChannelMap(device=mag1, output='WRITE_PV_1', read_fun=read_device, write_fun=write_device)
chm = ChannelMapper(maps=[m])
sim.add_device(mag1, period=10.0)
sim.add_mapper(chm)

mag2 = RealisticMagnet(name='mag2', value=0, low=-20.0, high=20.0, noise=0.1, resolution=None, model='instant')
m = ChannelMap(device=mag2, output='READ_PV_1', read_fun=read_device)
chm = ChannelMapper(maps=[m])
sim.add_device(mag2, period=10.0)
sim.add_mapper(chm)

osc = Oscillator(name='osc', period = 20.0, amplitude = 2.0)
m = ChannelMap(device=osc, output='GENERIC_PV_1', read_fun=read_device)
chm = ChannelMapper(maps=[m])
sim.add_device(osc, period=5.0)
sim.add_mapper(chm)

sim.start_update_thread()
print

[DEBUG] [     MainThread] [12:58:55         core.py 129] [SimulationEngine] Added device mag1 with update period of 10.0
[DEBUG] [     MainThread] [12:58:55         core.py 144] [SimulationEngine] Added mapper for outputs ['WRITE_PV_1'] from devices ['mag1']
[DEBUG] [     MainThread] [12:58:55         core.py 129] [SimulationEngine] Added device mag2 with update period of 10.0
[DEBUG] [     MainThread] [12:58:55         core.py 144] [SimulationEngine] Added mapper for outputs ['READ_PV_1'] from devices ['mag2']
[DEBUG] [     MainThread] [12:58:55         core.py 129] [SimulationEngine] Added device osc with update period of 5.0
[DEBUG] [     MainThread] [12:58:55         core.py 144] [SimulationEngine] Added mapper for outputs ['GENERIC_PV_1'] from devices ['osc']
[DEBUG] [     MainThread] [12:58:55         core.py 200] [SimulationEngine] Starting poll thread
[DEBUG] [sim_engine_poll] [12:58:55         core.py 168] [SimulationEngine] Hello from simulation poll thread (id 26748)


<function print>

In [3]:
sim.channels, sim.devices

(['WRITE_PV_1', 'READ_PV_1', 'GENERIC_PV_1'],
 {'mag1': <pybeamtools.sim.devices.RealisticMagnet at 0x1b093f06070>,
  'mag2': <pybeamtools.sim.devices.RealisticMagnet at 0x1b093f06250>,
  'osc': <pybeamtools.sim.devices.Oscillator at 0x1b093f06310>})

In [4]:
from pybeamtools.controls.control_lib import ConnectionOptions
ao = pc.AcceleratorOptions(connection_settings=ConnectionOptions(network='dummy'))
acc = pc.Accelerator(options=ao, ctx=sim)

[INFO ] [     MainThread] [12:58:55  control_lib.py  49] [Accelerator] Control lib init
[INFO ] [     MainThread] [12:58:55      network.py 157] [SimConnectionManager] Creating dummy connection manager
[INFO ] [     MainThread] [12:58:55  control_lib.py  71] [Accelerator] Startup finished


In [5]:
# Add PVs
from pybeamtools.controls.network import SimPV, PVOptions, PVAccess

write_triggers = ['WRITE_PV_1']
write_pv_1 = SimPV(PVOptions(name='WRITE_PV_1', low=-5.0, high=5.0, security=PVAccess.READWRITE))

read_triggers = ['READ_PV_1']
read_pv_1 = SimPV(PVOptions(name='READ_PV_1', security=PVAccess.READONLY))

generic_pv_1 = SimPV(PVOptions(name='GENERIC_PV_1', security=PVAccess.READONLY))

pv_list = ['GENERIC_PV_1'] + read_triggers + write_triggers
print(pv_list)

['GENERIC_PV_1', 'READ_PV_1', 'WRITE_PV_1']


In [6]:
acc.add_pv_object([read_pv_1, write_pv_1, generic_pv_1])
sim.schedule_update()
print()

[DEBUG] [     MainThread] [12:58:55      network.py 184] [SimConnectionManager] Adding 3 PV objects
[DEBUG] [     MainThread] [12:58:55         core.py 157] [SimulationEngine] Created subscription for channel READ_PV_1
[DEBUG] [     MainThread] [12:58:55         core.py 157] [SimulationEngine] Created subscription for channel WRITE_PV_1
[DEBUG] [     MainThread] [12:58:55         core.py 157] [SimulationEngine] Created subscription for channel GENERIC_PV_1


In [7]:
# This fails because of PV limits (no interlocks)
write_pv_1.write(5.5)
print()

InvalidWriteError: Value 5.5 above bounds (-5.0|5.0)

In [8]:
# This is allowed
write_pv_1.write(4.5)
print()

[DEBUG] [     MainThread] [12:58:58  control_lib.py 157] [Accelerator] Individual PV validation passed
[INFO ] [     MainThread] [12:58:58  control_lib.py 160] [Accelerator] Write on (['WRITE_PV_1']) triggered (0) of (0) interlocks
[DEBUG] [     MainThread] [12:58:58  control_lib.py 163] [Accelerator] No interlocks triggered, skip polling stage
[DEBUG] [     MainThread] [12:58:58         core.py 235] [SimulationEngine] Writing channel WRITE_PV_1


In [9]:
# Add an interlock to further constraint PVs
from pybeamtools.controls.interlocks import LimitInterlock, LimitInterlockOptions
interlock_limits = {'WRITE_PV_1':(-1.0,1.0)}

lopt = LimitInterlockOptions(pv_list=pv_list,
                            read_events=read_triggers,
                            write_events=pv_list,
                            limits=interlock_limits)
ilock = LimitInterlock(options=lopt)
acc.add_interlock(ilock)
print()

[DEBUG] [     MainThread] [12:58:59  distributed.py  97] [ProcessManager] Interlock worker started on PID 31576
[INFO ] [     MainThread] [12:58:59  control_lib.py 221] [Accelerator] Interlock (29705575-c49c-4500-89d6-159d0e960417) with PV list (['GENERIC_PV_1', 'READ_PV_1', 'WRITE_PV_1']) added


In [10]:
# Same write now fails due to interlocks
write_pv_1.write(4.5)
print()

[INFO ] [     MainThread] [12:59:00      logging.py  56] [pybeamtools.utils.logging] Starting worker logging finished


[DEBUG] [     MainThread] [12:59:00  control_lib.py 157] [Accelerator] Individual PV validation passed
[INFO ] [     MainThread] [12:59:00  control_lib.py 160] [Accelerator] Write on (['WRITE_PV_1']) triggered (1) of (1) interlocks
[DEBUG] [     MainThread] [12:59:00  control_lib.py 184] [Accelerator] Polling process manager with data packages of sizes ([3])
[DEBUG] [     MainThread] [12:59:00  control_lib.py 185] [Accelerator] data_packages=[{'trigger_pvs': {'WRITE_PV_1': 4.5}, 'data': {'READ_PV_1': array([-0.02338869]), 'GENERIC_PV_1': array([0.13822901]), 'WRITE_PV_1': array([0.10630662])}, 'timestamps': None}]
[DEBUG] [     MainThread] [12:59:00  distributed.py 119] [ProcessManager] Sent message {'trigger_pvs': {'WRITE_PV_1': 4.5}, 'data': {'READ_PV_1': array([-0.02338869]), 'GENERIC_PV_1': array([0.13822901]), 'WRITE_PV_1': array([0.10630662])}, 'timestamps': None} (id 485295) to interlock (29705575-c49c-4500-89d6-159d0e960417)
[INFO ] [     MainThread] [12:59:00  distributed.py  

InterlockWriteError: Write on (['WRITE_PV_1']) of ([4.5]) failed due to interlock rejection

In [11]:
# Smaller write is allowed
write_pv_1.write(0.5)
print()

[DEBUG] [     MainThread] [12:59:02  control_lib.py 157] [Accelerator] Individual PV validation passed
[INFO ] [     MainThread] [12:59:02  control_lib.py 160] [Accelerator] Write on (['WRITE_PV_1']) triggered (1) of (1) interlocks
[DEBUG] [     MainThread] [12:59:02  control_lib.py 184] [Accelerator] Polling process manager with data packages of sizes ([3])
[DEBUG] [     MainThread] [12:59:02  control_lib.py 185] [Accelerator] data_packages=[{'trigger_pvs': {'WRITE_PV_1': 0.5}, 'data': {'READ_PV_1': array([-0.02338869]), 'GENERIC_PV_1': array([1.99516599]), 'WRITE_PV_1': array([0.10630662])}, 'timestamps': None}]
[DEBUG] [     MainThread] [12:59:02  distributed.py 119] [ProcessManager] Sent message {'trigger_pvs': {'WRITE_PV_1': 0.5}, 'data': {'READ_PV_1': array([-0.02338869]), 'GENERIC_PV_1': array([1.99516599]), 'WRITE_PV_1': array([0.10630662])}, 'timestamps': None} (id 541357) to interlock (29705575-c49c-4500-89d6-159d0e960417)
[DEBUG] [     MainThread] [12:59:02  distributed.py 1

In [12]:
# Another interlock to block all writes when generic oscillator PV < 0
lopt2 = LimitInterlockOptions(pv_list=pv_list,
                            read_events=[],
                            write_events=write_triggers,
                            limits={'GENERIC_PV_1':(-10.0,0.0)},
                            block_all_writes=True)
ilock2 = LimitInterlock(options=lopt2)
acc.add_interlock(ilock2)

[DEBUG] [     MainThread] [12:59:03  distributed.py  97] [ProcessManager] Interlock worker started on PID 25536
[INFO ] [     MainThread] [12:59:03  control_lib.py 221] [Accelerator] Interlock (10a6ec92-dfbd-4260-b291-a74e85e55039) with PV list (['GENERIC_PV_1', 'READ_PV_1', 'WRITE_PV_1']) added


In [13]:
write_pv_1.write(0.5)
print()

[DEBUG] [     MainThread] [12:59:04  control_lib.py 157] [Accelerator] Individual PV validation passed
[INFO ] [     MainThread] [12:59:04  control_lib.py 160] [Accelerator] Write on (['WRITE_PV_1']) triggered (2) of (2) interlocks
[DEBUG] [     MainThread] [12:59:04  control_lib.py 184] [Accelerator] Polling process manager with data packages of sizes ([3, 3])
[DEBUG] [     MainThread] [12:59:04  control_lib.py 185] [Accelerator] data_packages=[{'trigger_pvs': {'WRITE_PV_1': 0.5}, 'data': {'READ_PV_1': array([-0.02338869]), 'GENERIC_PV_1': array([1.99516599]), 'WRITE_PV_1': array([0.10630662])}, 'timestamps': None}, {'trigger_pvs': {'WRITE_PV_1': 0.5}, 'data': {'READ_PV_1': array([-0.02338869]), 'GENERIC_PV_1': array([1.99516599]), 'WRITE_PV_1': array([0.10630662])}, 'timestamps': None}]
[DEBUG] [     MainThread] [12:59:04  distributed.py 119] [ProcessManager] Sent message {'trigger_pvs': {'WRITE_PV_1': 0.5}, 'data': {'READ_PV_1': array([-0.02338869]), 'GENERIC_PV_1': array([1.9951659

InterlockWriteError: Write on (['WRITE_PV_1']) of ([0.5]) failed due to interlock rejection

In [14]:
from pybeamtools.controls.interlocks import RatelimitInterlock,RatelimitInterlockOptions
lopt3 = RatelimitInterlockOptions(pv_list=pv_list,
                            read_events=[],
                            write_events=write_triggers,
                            min_delay=0.5)
ilock3 = RatelimitInterlock(options=lopt3)
acc.add_interlock(ilock3)

[DEBUG] [     MainThread] [12:59:08  distributed.py  97] [ProcessManager] Interlock worker started on PID 27332
[INFO ] [     MainThread] [12:59:08  control_lib.py 221] [Accelerator] Interlock (49d9a24f-9566-4130-a738-18577994b599) with PV list (['GENERIC_PV_1', 'READ_PV_1', 'WRITE_PV_1']) added


[INFO ] [     MainThread] [12:59:09      logging.py  56] [pybeamtools.utils.logging] Starting worker logging finished


In [17]:
write_pv_1.write(0.5)

[DEBUG] [     MainThread] [12:59:14  control_lib.py 157] [Accelerator] Individual PV validation passed
[INFO ] [     MainThread] [12:59:14  control_lib.py 160] [Accelerator] Write on (['WRITE_PV_1']) triggered (3) of (3) interlocks
[DEBUG] [     MainThread] [12:59:14  control_lib.py 184] [Accelerator] Polling process manager with data packages of sizes ([3, 3, 3])
[DEBUG] [     MainThread] [12:59:14  control_lib.py 185] [Accelerator] data_packages=[{'trigger_pvs': {'WRITE_PV_1': 0.5}, 'data': {'READ_PV_1': array([0.03684466]), 'GENERIC_PV_1': array([-1.9953702]), 'WRITE_PV_1': array([0.41808176])}, 'timestamps': None}, {'trigger_pvs': {'WRITE_PV_1': 0.5}, 'data': {'READ_PV_1': array([0.03684466]), 'GENERIC_PV_1': array([-1.9953702]), 'WRITE_PV_1': array([0.41808176])}, 'timestamps': None}, {'trigger_pvs': {'WRITE_PV_1': 0.5}, 'data': {'READ_PV_1': array([0.03684466]), 'GENERIC_PV_1': array([-1.9953702]), 'WRITE_PV_1': array([0.41808176])}, 'timestamps': None}]
[DEBUG] [     MainThread]

InterlockWriteError: Write on (['WRITE_PV_1']) of ([0.5]) failed due to interlock rejection

[DEBUG] [     MainThread] [12:59:14   interlocks.py 204] [RatelimitInterlock] Interlock (49d9a24f-9566-4130-a738-18577994b599) write callback evaluation with pv_list=['GENERIC_PV_1', 'READ_PV_1', 'WRITE_PV_1'] read_events=[] write_events=['WRITE_PV_1'] priority=0 min_delay=0.5 {'READ_PV_1': array([0.03684466]), 'GENERIC_PV_1': array([-1.9953702]), 'WRITE_PV_1': array([0.41808176])}


In [18]:
acc.interlocks

In [26]:
process = acc.pm.process_map[ilock2.uuid]

In [27]:
process, process.pid

(<SpawnProcess name='SpawnProcess-2' pid=25536 parent=26308 started>, 25536)

In [29]:
# We fake a crash by killing process with a syscall
import psutil
p = psutil.Process(process.pid)
p

psutil.Process(pid=25536, name='python.exe', status='running', started='12:59:03')

In [ ]:
process.terminate()

In [ ]:
acc.pm.processes

In [ ]:
acc.options.dict()

In [22]:
write_pv_1.write(0.5)

[DEBUG] [     MainThread] [13:01:34  control_lib.py 157] [Accelerator] Individual PV validation passed
[INFO ] [     MainThread] [13:01:34  control_lib.py 160] [Accelerator] Write on (['WRITE_PV_1']) triggered (3) of (3) interlocks
[DEBUG] [     MainThread] [13:01:34  control_lib.py 184] [Accelerator] Polling process manager with data packages of sizes ([3, 3, 3])
[DEBUG] [     MainThread] [13:01:34  control_lib.py 185] [Accelerator] data_packages=[{'trigger_pvs': {'WRITE_PV_1': 0.5}, 'data': {'READ_PV_1': array([-0.03704671]), 'GENERIC_PV_1': array([-1.99567794]), 'WRITE_PV_1': array([0.45983416])}, 'timestamps': None}, {'trigger_pvs': {'WRITE_PV_1': 0.5}, 'data': {'READ_PV_1': array([-0.03704671]), 'GENERIC_PV_1': array([-1.99567794]), 'WRITE_PV_1': array([0.45983416])}, 'timestamps': None}, {'trigger_pvs': {'WRITE_PV_1': 0.5}, 'data': {'READ_PV_1': array([-0.03704671]), 'GENERIC_PV_1': array([-1.99567794]), 'WRITE_PV_1': array([0.45983416])}, 'timestamps': None}]


ControlLibException: Interlock process_read terminated unexpectedly